In [1]:
import numpy as np
import pandas as pd
pd.set_option("display.max_row", 50)
pd.set_option("display.max_column", 50)

from sklearn.model_selection import train_test_split

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

from sklearn.model_selection import StratifiedKFold

from sklearn.ensemble import ExtraTreesClassifier

from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_val_score

import gc
import re
import os
import time
import random
import platform
import datetime as dt
from tqdm import tqdm
from itertools import combinations
from scipy.stats.mstats import gmean
import warnings; warnings.filterwarnings("ignore")

In [2]:
train = pd.read_csv('../data/trainset.csv')
test = pd.read_csv('../data/testset_final.csv')
submission = pd.read_csv('../data/sample_submission_final.csv')

In [3]:
# train test 겹치지 않는 열 삭제
train = train.drop(columns=['매니저최초가입일' , '매니저최초서비스일','매니저주소'])

In [4]:
# 매니저 생년만 반환
train['매니저생년월일'] = train['매니저생년월일'].apply(lambda x: int(str(x)[:-6]))

In [5]:
tr = pd.concat([train, test])

In [6]:
#To Fill
col1 = {'평수':'없음',
        '결재형태' : '신용카드',
        '고객가입일' : '2020-06-23',
        '반려동물':'없음',
        '부재중여부': '모름',
        '우선청소' :'없음',
        '매니저사용휴대폰': '안드로이드',
        '매니저이동방법': '대중교통',
        'CS교육이수여부': 0,
        '청소교육이수여부': 0}
#To drop
col2 = {'접수시각', '매칭성공여부'}

In [7]:
tr = tr.fillna(col1)
tr = tr.drop(col2, axis='columns')

In [8]:
# 접수 월,일 생성
tr['접수월'] = tr['접수일'].apply(lambda x: x[5:7]).astype(int)
tr['접수일일'] = tr['접수일'].apply(lambda x: x[-2:]).astype(int)

In [9]:
# 최초서비스일,서비스일자 datetime으로 변경
tr.최초서비스일 = pd.to_datetime(tr.최초서비스일)
tr.서비스일자 = pd.to_datetime(pd.to_datetime(tr.서비스일자))

In [10]:
# 사용자가 서비스를 이용한 기간 생성
tr['서비스이용기간'] = tr.서비스일자-tr.최초서비스일

In [11]:
tr['서비스이용기간'] = tr['서비스이용기간'].apply(lambda x: int(str(x)[:-14]))

In [12]:
def age(x):
    if x < 1960:
        return 1
    elif x < 1970:
        return 2
    elif x < 1980:
        return 3
    elif x < 1990:
        return 4
    else:
        return 5

In [13]:
# 매니저 생년으로 연령대 생성
tr['매니저연령대'] = tr['매니저생년월일'].apply(lambda x: age(x))

In [14]:
def pet(x):
    if x == '없음':
        return 1
    else:
        return 0

In [15]:
# 반려동물여부 생성
tr['반려동물여부'] = tr['반려동물'].apply(lambda x: pet(x))

In [16]:
# 매니저 매칭에 영향을 미치는 여러 특성을 고려하여 점수화 함
tr['종합지수'] = tr['부재중서비스가능여부'] - tr['쿠폰사용여부'] + tr['CS교육이수여부'] + tr['청소교육이수여부'] + tr['추천인여부'] + tr['반려동물여부']

In [17]:
# 서비스 시작 시간
tr['서비스시작시간_2'] = tr['서비스시작시간'].apply(lambda x: int(x[:-6]))

In [18]:
def hour(x):
    if x <=12 and x >= 6:
        return 0
    else:
        return 1

In [19]:
# 서비스 시작 시간 범주화
tr['서비스시간대'] = tr['서비스시작시간_2'].apply(lambda x: hour(x))

In [20]:
tr = tr.drop(columns = '서비스시작시간_2', axis=1)

In [21]:
def house_size(x):
    if x == '10평대':
        return 1
    elif x == '20평대':
        return 2
    elif x == '30평대':
        return 3
    elif x == '모름':
        return 0
    else:
        return 4

In [22]:
# 평수 범주화
tr['평수_1'] = tr['평수'].apply(lambda x: house_size(x))

In [23]:
def house_type(x):
    if x == '일반주택':
        return 1
    else:
        return 2

In [24]:
# 주거형태 범주화
tr['주거형태_1'] = tr['주거형태'].apply(lambda x: house_type(x))

In [25]:
# 주거 가중치
tr['주거가중치'] = tr['주거형태_1'] + tr['평수_1']

In [26]:
tr = tr.drop(columns =['평수_1','주거형태_1'], axis=1)

In [27]:
tr = tr.replace('수도권', '서울/경기/인천')
tr = tr.replace('천안/아산', '충남/천안/아산')

In [28]:
# 서비스 주소 범주화
region = []
for i in tr.서비스주소 :
    region.append(i[:2])

tr['서비스주소_광역'] = region

In [29]:
# 서비스 시작,종로 시간 datetime으로 변경
tr['서비스시작시간'] = pd.to_datetime(pd.to_datetime(tr.서비스시작시간))
tr['서비스종료시간'] = pd.to_datetime(pd.to_datetime(tr.서비스종료시간))

In [30]:
# 서비스 시간
tr['서비스시간'] = tr['서비스종료시간'].dt.hour - tr['서비스시작시간'].dt.hour 

In [31]:
# 접수일 datetime
tr['접수일'] = pd.to_datetime(pd.to_datetime(tr['접수일']))

In [32]:
# 서비스대기기간
tr['서비스대기기간'] = (tr['최초서비스일'] - tr['접수일']).apply(lambda x: int(str(x)[:-14]))

In [33]:
# 접수요일
tr['접수요일'] = tr['접수일'].dt.weekday

In [34]:
# 매니저 현재 나이
tr['매니저나이'] = 2021 - tr['매니저생년월일']

In [35]:
train['전체회차'].describe()

count    23009.000000
mean         4.642357
std          2.323255
min          1.000000
25%          4.000000
50%          4.000000
75%          4.000000
max         30.000000
Name: 전체회차, dtype: float64

In [36]:
# 장기 단기 고객
def service(x):
    if x >= 5:
        return '장기'
    else:
        return '단기'

train['서비스이용기준'] = train['전체회차'].apply(lambda x: service(x))

In [37]:
# 고객 가입일 datetime
tr['고객가입일'] = pd.to_datetime(pd.to_datetime(tr.고객가입일))

In [38]:
tr['고객가입일'] = tr['최초서비스일'] - tr['고객가입일']

In [39]:
tr['고객가입일'] = tr['고객가입일'].apply(lambda x : int(str(x)[:-14]))

In [40]:
tr = tr.astype({'매니저나이': 'object'})
tr = tr.astype({'매니저연령대': 'object'})

In [41]:
# 시간 열 제거
col3 = {'서비스시작시간','서비스종료시간'}      
tr = tr.drop(col3, axis='columns')

In [42]:
# 범주형 Feature로 변환
tl = ['접수일', '최초서비스일', '서비스일자', '고객가입일', '매니저성별', '매니저생년월일', '접수월', '접수일일', '접수요일']
for i in tl:
    tr[i] = tr[i].astype('object')

In [43]:
tr.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27253 entries, 0 to 4243
Data columns (total 39 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   SEQ         27253 non-null  object 
 1   접수일         27253 non-null  object 
 2   장기서비스여부     27253 non-null  int64  
 3   최초서비스일      27253 non-null  object 
 4   전체회차        27253 non-null  int64  
 5   현재회차        27253 non-null  int64  
 6   서비스일자       27253 non-null  object 
 7   기존고객여부      27253 non-null  int64  
 8   결재형태        27253 non-null  object 
 9   서비스주소       27253 non-null  object 
 10  주거형태        27253 non-null  object 
 11  평수          27253 non-null  object 
 12  고객가입일       27253 non-null  object 
 13  반려동물        27253 non-null  object 
 14  부재중여부       27253 non-null  object 
 15  우선청소        27253 non-null  object 
 16  쿠폰사용여부      27253 non-null  int64  
 17  매니저생년월일     27253 non-null  object 
 18  매니저성별       27253 non-null  object 
 19  매니저사용휴대폰    27253 non-null

In [44]:
tr.서비스주소_광역.value_counts()

충남    18164
서울     3558
강원     3325
광주      825
부산      805
경기      533
경남       43
Name: 서비스주소_광역, dtype: int64

In [45]:
# 평수 주거형태 반려동물 매니저사용휴대폰 서비스주소_광역

In [46]:
train_data = tr[0:len(train)]
train_label = train.매칭성공여부
train_data['매칭성공여부'] = train_label

In [47]:
# categorical_cols = ['평수' '주거형태' '반려동물' '매니저사용휴대폰' '서비스주소_광역']

In [48]:
def one_hot_encoder(df, categorical_columns=None, nan_as_category=True):
    """Create a new column for each categorical value in categorical columns. """
    original_columns = list(df.columns)
    if not categorical_columns:
        categorical_columns = [col for col in df.columns if df[col].dtype == 'object'] #None로 정의
    df = pd.get_dummies(df, columns=categorical_columns, dummy_na=nan_as_category) #더미변수화
    categorical_columns = [c for c in df.columns if c not in original_columns]
    return df, categorical_columns

In [49]:
categorical_columns=None
nan_as_category=True
original_columns = list(train.columns)
categorical_columns = [col for col in tr.columns if tr[col].dtype == 'object'] # object변수 뽑기

In [50]:
categorical_columns

['SEQ',
 '접수일',
 '최초서비스일',
 '서비스일자',
 '결재형태',
 '서비스주소',
 '주거형태',
 '평수',
 '고객가입일',
 '반려동물',
 '부재중여부',
 '우선청소',
 '매니저생년월일',
 '매니저성별',
 '매니저사용휴대폰',
 '매니저이동방법',
 '근무가능지역',
 '접수월',
 '접수일일',
 '매니저연령대',
 '서비스주소_광역',
 '접수요일',
 '매니저나이']

In [51]:
tr1 = tr.drop('SEQ', axis=1)

In [52]:
tr1 = pd.get_dummies(tr1)

In [53]:
tr = pd.concat([tr.SEQ,tr1],axis=1)

In [54]:
train_data = tr.iloc[:len(train),:]
test_data = tr.iloc[len(train):,:]

In [55]:
train_data = train_data.drop('SEQ', axis='columns')
test_data = test_data.drop('SEQ', axis='columns')

In [56]:
train_label = train.매칭성공여부

In [57]:
seed = 42

In [58]:
from imblearn.over_sampling import *
from imblearn.under_sampling import *
from imblearn.combine import *
smote = SMOTETomek(sampling_strategy=0.5, random_state=seed)
train_data_over,train_label_over = smote.fit_resample(train_data,train_label)

In [59]:
# from imblearn.combine import SMOTEENN
# smt = SMOTEENN(sampling_strategy=0.5, random_state=42)
# # X, y = smt.fit_sample(X, y)
# train_data_over,train_label_over = smt.fit_resample(train_data,train_label)

In [60]:
train_data_over.reset_index(drop = True, inplace=True)
train_label_over.reset_index(drop = True, inplace=True)

In [61]:
train_data_over.to_csv('../data/WannaVeteran_train.csv',index=False)
test_data.to_csv('../data/WannaVeteran_test.csv',index=False)
train_label_over.to_csv('../data/WannaVeteran_train_label.csv',index=False)

In [62]:
train_data_over = pd.read_csv('../data/WannaVeteran_train.csv')
test_data = pd.read_csv('../data/WannaVeteran_test.csv')
train_label = pd.read_csv('../data/WannaVeteran_train_label.csv')

In [63]:
n_it = 60
random_state=42
Kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=random_state)

# 모델 훈련 함수 정의
def return_fitted_model(model, train, target):
    scores = []
    models = []
    for iter_count, (train_idx, valid_idx) in enumerate(Kfold.split(train, target)):

        X_train, X_valid = train[train_idx], train[valid_idx]
        y_train, y_valid = target[train_idx], target[valid_idx]

        model.fit(X_train, y_train)

        pred = model.predict(X_valid)
        ra_score = roc_auc_score(y_valid, pred)
        scores.append(ra_score)
        models.append(model)
    return model, np.mean(scores)

# ex: extra_reg, extra_reg_score = return_fitted_model(extra_reg, train, target)
# print(f'모델의 평균 성능:  {extra_reg_score:.3f}')

In [64]:
model = ExtraTreesClassifier(random_state=seed, n_jobs=-1)

In [65]:
model, avg_score = return_fitted_model(model, train_data_over.values, train_label_over.values)
print('score: ',avg_score)

score:  0.9526012376681855


In [66]:
model.predict_proba(test_data)[:,1]

array([0.05, 0.  , 1.  , ..., 0.  , 0.  , 0.22])

In [67]:
pd.DataFrame({'SEQ': test.SEQ , 
              'pred': model.predict_proba(test_data)[:,1]}).to_csv("../submission/WannaVeteran_submission.csv", index=False)